![1](ATLASOD.gif)

# Pirma pamoka

Šioje pamokoje mes:
* Susipažinsime su dalelių fizikos CERN ROOT biblioteka
* Išmoksime įskaityti failą su protonų susidūrimų duomenimis
* Atliksime imtis reikalingas studijuoti $Z\to\ell\ell$ procesą
* Apskaičiuosime leptonų sistemos keturmatį vektorių ir nupiešime masės spektro histogramą

## Z bozono skilimas

Z bozonas yra atsakingas už silpnąją sąveiką be krūvio perdavimo. Ši dalelė skyla beveik akimirksniu, įmanoma užfiksuoti tik jos skeveldros. Lengviausiai matomi skilimai yra į miuonų porą $Z^0\to\mu^+\mu^-$ arba elektronų porą $Z^0\to e^+e^-$:
![3](Zskilimas.png)

In [ ]:
# Įkraunama ROOT biblioteka
import ROOT
%jsroot on

# matematinių operacijų biblioteka
import math 

In [ ]:
# Duomenų analizę pradedame pasitelkdami simuliacija. Atidarome ROOT failą
# Šiame faile talpinami tik simuliuoti Z bozono įvykiai
nuoroda = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/mc_363356.ZqqZll.2lep.root"
failas = ROOT.TFile.Open(nuoroda)
# Iš failo nuskaitome "medį" (tree) su susidūrimų duomenimis
medis = failas.Get("mini")

In [ ]:
# Atliekame paprastas operacijas su nuskaitytu medžiu: išvedame susidūrimų skaičių į ekraną
n_viso = medis.GetEntries()
print(f"Faile rasta {n_viso} įvykių")

In [ ]:
# Pažiūrėkime kokią informaciją galime rasti savo medyje
medis.Print()

# Pastaba: norint filtruoti šakas galima naudoti "wildcard", pvz:
# medis.Print("lep*")

In [ ]:
# Kad nupiešti vieną iš medžio šakų galima pasinaudoti Draw komanda
# Tačiau pirma reikia nupiešti "drobę" (canvas)
drobė = ROOT.TCanvas()
drobė.Draw()

# Nupieškime histogramą, rodančią leptonų skaičių įvykyje
medis.Draw("lep_n")
# Gautas grafikas yra interaktyvus (užveskite pele)

### Kiekviena medžio "šaka" aprašo vis kitokias įvykio savybes. Poros iš jų paaiškinimas:
`runNumber` - duomenų rinkimo periodo numeris  
`eventNumber` - unikalus įvykio identifikatorius  
`mcWeight` - simuliuoti įvykio svoriai. Reikalingi norint palyginti simuliuotus duomenis su tikrais duomenimis  
`lep_` - leptonų (elektronų ir miuonų) duomenys. Pastebėkime, kad šie duomenys talpinami vektoriuje (`vector<float>`). Taip yra dėl to, kad kiekviename įvykyje susidaro kintamas kiekis leptonų  
`jet_` - hadroninių čiurkšlių (jet) duomenys  
`photon_` - fotonų duomenys  

## Užduotys

__Užduotis 1:__ nupieškite leptonų krūvio (`lep_charge`) histogramą. Ar gavote tai, ko tikėjotės?  

In [ ]:
# Užduotis 1 atliekama čia
medis.Draw("...")

__Užduotis 2:__ nupieškite leptono tipo (`lep_type`) histogramą. Kiekviena dalelė simuliacijoje turi savo paskirtą PDG numerį. Kokius leptonus turime? Daugiau informacijos: https://particle.wiki/wiki/PDG_particle_numbering_scheme

In [ ]:
# Užduotis 2 atliekama čia
medis.Draw("...")

Histogramos labai naudingos statistinei (daugelio įvykių) analizei. Tačiau pirma pasižiūrėkime nuosekliau, kaip atrodo pasirinktas vienas įvykis

In [ ]:
# Naudojame for ciklą atlikti medžio iteraciją, tačiau sustojame po pirmo įvykio 
for įvykis in medis:
    print("Įskaitėme vieną įvykį...")
    
    # Atliekame antrą for ciklą kad atlikti vienos šakos iteraciją vienam įvykiui
    for i, pt in enumerate(įvykis.lep_pt):
        print(f"Leptono {i} skersinis judesio kiekis: {pt:.1f} MeV")
    break

## Pilno įvykio vizualizacija
Apačioje matomas vienas \$Z\to e^+e^-$ kandidatinis įvykis, užfiksuotas ATLAS detektoriaus 2022 m. liepos mėn. Vaizdiškai išanalizuoti milijardus tokių įvykių neįmanoma, taigi turime pasitelkti kompiuteriniais skaičiavimais.
![2](event_display_Zee_Run_427394_Event10631430_5Jul2022.png)

## Demonstracija: Z bozono rezonanso rekonstravimas
###  Atliksime skaičiavimą keliais etapais:
1. Tam tikrame įvykyje identifikuoti abu leptonų kandidatus
2. Apskaičiuoti leptonų sistemos invariantinę masę
3. Užpildyti leptonų masės histogramą
4. Pereiti prie sekančio įvykio ir grįžti prie 1 punkto

In [ ]:
# Sukuriame histogramą (H), vienos-dimensijos (1) ir "dvigubo tikslumo" (D) 
# Paskutiniai trys argumentai apibrėžia intervalų skaičių
# bei histogramos apatinę ir viršutinę ribas
# pirmas argumentas: pavadinimas
# antras argumentas: pavadinimas, x-ašies pavadinimas, y-ašies pavadinimas
masės_histograma = ROOT.TH1D("masė", "masė; Leptonų sistemos masė [GeV]; įvykių skaičius", 
                             100, 60, 120)

# Skaičiuojame, kiek įvykių pasirinkome iš visų esamų
n = 0

# Sukurkime po keturmatį vektorių abiems leptonų kandidatams
pirmas_leptonas = ROOT.TLorentzVector()
antras_leptonas = ROOT.TLorentzVector()

for įvykis in medis:
    # Įsitikiname, kad yra bent jau du leptonai
    # Kitaip praleidžiame šį įvykį
    if įvykis.lep_n < 2:
        continue

    # Leptonų krūviai privalo būti skirtingi
    if (įvykis.lep_charge[0] == įvykis.lep_charge[1]):
        continue 

    # Užregistruojame leptonų savybes reikalingas užpildyti keturmatį vektorių
    pirmo_pt = įvykis.lep_pt[0]/1000.
    antro_pt = įvykis.lep_pt[1]/1000.
    pirmo_E = įvykis.lep_E[0]/1000.
    antro_E = įvykis.lep_E[1]/1000.

    # Padalijome judesio kiekį (impulsą) ir energiją iš 1000, kad paversti vienetus iš MeV į GeV
    pirmo_phi = įvykis.lep_phi[0]
    antro_phi = įvykis.lep_phi[1]
    pirmo_eta = įvykis.lep_eta[0]
    antro_eta = įvykis.lep_eta[1]

    # ATLAS naudoja ne (px, py, pz, E) atskaitos sistemą, o (pT, eta, phi, E)
    # Daugiau informacijos apie tai apačioje (prie klausimų ir užduočių)
    pirmas_leptonas.SetPtEtaPhiE(pirmo_pt, pirmo_eta, pirmo_phi, pirmo_E)
    antras_leptonas.SetPtEtaPhiE(antro_pt, antro_eta, antro_phi, antro_E)

    # Sudedami judesio kiekius gauname sistemos pilną (keturmatį) judesio kiekį:
    # p(Z) = p(l1) + p(l2)
    sistema = pirmas_leptonas + antras_leptonas
    
    # Kad gauti sistemos masę labai paprasta:
    masė = sistema.M()

    # Užpildome histogramą
    masės_histograma.Fill(masė)

    # Išvedame į ekraną kas 10000 įvykių
    if n % 10000 == 0:
        print(f"Pasirinkta {n} įvykių")
        
    # Kad pagreitinti skaičiavimą, sustojame ties 100k pasirinktų įvykių
    # Jeigu norite padidinti duomenų kiekį, galite ištrinti dvi sekančias eilutes
    if n == 100000:
        print("Baigiame")
        break

    n += 1

print(f"Iš viso pasirinkta {n}/{n_viso} įvykių")

In [ ]:
# Nupieškime drobę ir histogramą
drobė.Draw()
masės_histograma.SetLineColor(ROOT.kRed)
masės_histograma.Draw()

## Klausimai ir užduotys:
Sveikiname, turėtų aiškiai matytis Z bozono rezonansas, skylant į du leptonus (miuonus arba elektronus)!

__Užduotis #3__: ar galite nupiešti dvi atskiras histogramas, vieną $Z\to\mu^+\mu^-$, o kitą $Z\to e^+e^-$ skilimui?  

In [ ]:
# Užduotis 3 atliekama čia

# Sukurkite dvi histogramas
# --- jūsų kodas eina čia ---
mumu_histograma = ...
ee_histograma = ...

# Skaičiuojame, kiek įvykių pasirinkome (iš visų įmanomų)
n = 0

# Sukurkime po keturmatį vektorių abiems leptonų kandidatams
pirmas_leptonas = ROOT.TLorentzVector()
antras_leptonas = ROOT.TLorentzVector()

for įvykis in medis:
    # Įsitikiname, kad yra bent jau du leptonai
    # Kitaip praleidžiame šį įvykį
    if įvykis.lep_n < 2:
        continue

    # Leptonų krūviai privalo būti skirtingi
    if (įvykis.lep_charge[0] == įvykis.lep_charge[1]):
        continue 
        
    # Patikrinkite, ar leptonai yra vienodi (toks pat tipas "type")
    # Jeigu nevienodi, praleiskite įvykį ("continue")
    # --- jūsų kodas eina čia ---
    if ...:
        continue

    # Užregistruojame leptonų savybes reikalingas užpildyti keturmatį vektorių
    pirmo_pt = įvykis.lep_pt[0]/1000.
    antro_pt = įvykis.lep_pt[1]/1000.
    pirmo_E = įvykis.lep_E[0]/1000.
    antro_E = įvykis.lep_E[1]/1000.

    pirmo_phi = įvykis.lep_phi[0]
    antro_phi = įvykis.lep_phi[1]
    pirmo_eta = įvykis.lep_eta[0]
    antro_eta = įvykis.lep_eta[1]

    pirmas_leptonas.SetPtEtaPhiE(pirmo_pt, pirmo_eta, pirmo_phi, pirmo_E)
    antras_leptonas.SetPtEtaPhiE(antro_pt, antro_eta, antro_phi, antro_E)

    # p(Z) = p(l1) + p(l2)
    sistema = pirmas_leptonas + antras_leptonas
    masė = sistema.M()

    # Užpildome tinkamą histogramą priklausomai nuo leptonų tipo
    # --- jūsų kodas eina čia ---

    if ...:
        mumu_histograma.Fill(masė)
    else:
        ee_histograma.Fill(masė)

    # Išvedame į ekraną kas 10000 įvykių
    if n % 10000 == 0:
        print(f"Pasirinkta {n} įvykių")
        
    # Kad pagreitinti skaičiavimą, sustojame ties 100k pasirinktų įvykių
    # Jeigu norite padidinti duomenų kiekį, galite ištrinti dvi sekančias eilutes
    if n == 100000:
        print("Baigiame")
        break

    n += 1

print(f"Iš viso pasirinkta {n}/{n_viso} įvykių")

In [ ]:
# Nupieškime drobę ir histogramą
drobė.Draw()
mumu_histograma.SetLineColor(ROOT.kRed)
mumu_histograma.Draw()

# Antrą histogramą virš pirmos galima nupiešti toje pačioje drobėje, pasinaudojus `SAME` komanda
ee_histograma.Draw("SAME")

__Užduotis #4__: Skaičiavome leptonų sistemos masę `ROOT` programos keturmačių vektorių bibliotekos pagalba. Pabandykite apskaičiuoti masę patys, pasinaudodami reliatyvistiniu energijos-judesio kiekio tvermės dėsniu, $E^2 = \vec{p}^2 + m^2$ (nekreipiame dėmesio į šviesos greičio c konstantas). Tai padarome suskaičiuodami leptonų sistemos (ir tuo pačiu Z bozono) invariantinę masę:  
$m^2 = E^2 - \vec{p}^2 = (E_{\ell_2} + E_{\ell_1})^2 - (\vec{p}_{\ell_1} + \vec{p}_{\ell_2})^2$  

__Pastaba:__ kad suskaičiuoti judesio kiekių vektorinę sumą, reikia atskirų $(p_x, p_y, p_z)$ projekcijų. tačiau medyje randamos tik $(p_\mathrm{T}, \eta, \phi)$ koordinatės. Galite pasinaudoti šiomis formulėmis, kad pereiti iš vienos koordinačių sistemos į kitą:  

$p_x = p_\mathrm{T} \cos{\phi}, \quad p_y = p_\mathrm{T} \sin{\phi}, \quad p_z = p_\mathrm{T} \sinh{\eta}.$

Apskaičiuoti trigonometrines ir hiperbolines funkcijas galima su python `math` biblioteka, pvz. `y = math.cos(x)`

In [ ]:
# Sukurkime po keturmatį vektorių abiems leptonų kandidatams
pirmas_leptonas = ROOT.TLorentzVector()
antras_leptonas = ROOT.TLorentzVector()

for įvykis in medis:
    if įvykis.lep_n < 2:
        continue

    if (įvykis.lep_charge[0] == įvykis.lep_charge[1]):
        continue 

    pirmo_pt = įvykis.lep_pt[0]/1000.
    antro_pt = įvykis.lep_pt[1]/1000.
    pirmo_E = įvykis.lep_E[0]/1000.
    antro_E = įvykis.lep_E[1]/1000.

    pirmo_phi = įvykis.lep_phi[0]
    antro_phi = įvykis.lep_phi[1]
    pirmo_eta = įvykis.lep_eta[0]
    antro_eta = įvykis.lep_eta[1]

    pirmas_leptonas.SetPtEtaPhiE(pirmo_pt, pirmo_eta, pirmo_phi, pirmo_E)
    antras_leptonas.SetPtEtaPhiE(antro_pt, antro_eta, antro_phi, antro_E)

    # Sudedami judesio kiekius gauname sistemos pilną (keturmatį) judesio kiekį:
    # p(Z) = p(l1) + p(l2)
    sistema = pirmas_leptonas + antras_leptonas
    
    # Kad gauti sistemos masę labai paprasta:
    masė_root = sistema.M()
    
    # Apskaičiuokite masę "rankiniu" būdu
    masė = ...
    
    print(f"Palyginame mases: {masė} mūsų apskaičiuota, o turėjo gautis {masė_root}")
    break